In [1]:
%pip install pycocotools
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
from pathlib import Path
from sklearn.model_selection import train_test_split
from pycocotools.coco import COCO
import csv

import torch, os, time
import torch.nn.functional as F
import torchvision
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision import datasets, transforms
from itertools import product
import copy
import os
from pathlib import Path

In [3]:
data_dir = Path('data/')
src_json = data_dir / "annotations.json"              # original 60-class annotations
dst_json = data_dir / "annotations_10cats.json"
out_csv  = data_dir / "image_labels.csv"


In [4]:
old_to_new = {
    0:2, 1:8, 2:8, 3:8, 4:0, 5:0, 6:1, 7:0, 8:2, 9:1, 10:2, 11:2, 12:2,
    13:3, 14:3, 15:3, 16:3, 17:3, 18:3, 19:3, 20:3, 21:0, 22:7, 23:1,
    24:0, 25:5, 26:1, 27:0, 28:2, 29:4, 30:3, 31:3, 32:3, 33:3, 34:3,
    35:3, 36:4, 37:0, 38:0, 39:0, 40:0, 41:0, 42:0, 43:0, 44:0, 45:0,
    46:7, 47:0, 48:0, 49:0, 50:2, 51:6, 52:2, 53:6, 54:0, 55:0, 56:3,
    57:7, 58:9, 59:9
}

grand_names = [
    "Plastic containers & bottles",
    "Glass",
    "Metal",
    "Paper & cardboard",
    "Plastic film / wrappers",
    "Food waste / organics",
    "Textiles & misc. items",
    "Polystyrene / foam",
    "Hazardous / special waste",
    "Other / unlabelled"
]

# ----- Load original TACO annotations -----
with open(src_json) as f:
    coco = json.load(f)

# ----- Determine a single grand category per image -----
image_to_cat = {}   # image_id -> grand_id
for ann in coco["annotations"]:
    img_id = ann["image_id"]
    grand_id = old_to_new[ann["category_id"]]
    # policy: choose the lowest grand_id if multiple present
    if img_id not in image_to_cat:
        image_to_cat[img_id] = grand_id
    else:
        image_to_cat[img_id] = min(image_to_cat[img_id], grand_id)

# ----- Build a simple table: image file name -> grand category -----
rows = []
for img in coco["images"]:
    img_id = img["id"]
    if img_id in image_to_cat:   # skip images with no annotations
        rows.append({
            "file_name": img["file_name"],
            "grand_id": image_to_cat[img_id],
            "grand_name": grand_names[image_to_cat[img_id]]
        })

# ----- Save to CSV -----
with open(out_csv, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["file_name", "grand_id", "grand_name"])
    writer.writeheader()
    writer.writerows(rows)

print(f"✓ Wrote {len(rows)} rows to {out_csv}")

✓ Wrote 1500 rows to data/image_labels.csv


In [16]:
# %pip install Pillow
# %pip install torch torchvision

# import pandas as pd
# from sklearn.model_selection import train_test_split
# from PIL import Image
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms
# import torch

# data_dir   = Path("data")
# images_dir = data_dir / "images"
# csv_file   = data_dir / "image_labels.csv"

# # ---------- Train / Test split ----------
# df = pd.read_csv(csv_file)
# train_df, test_df = train_test_split(
#     df,
#     test_size=0.2,        # 80% train, 20% test
#     random_state=42,
#     shuffle=True
# )
# print(f"Train: {len(train_df)}  Test: {len(test_df)}")

# # (Optional) save the splits
# train_df.to_csv(data_dir / "train_labels.csv", index=False)
# test_df.to_csv(data_dir / "test_labels.csv", index=False)

# # ---------- Transform: resize shortest side to 1024, keep full image ----------
# train_transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
#     transforms.Resize(1024),    # shortest side = 1024 px, keep aspect ratio
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
# ])

# test_transform = transforms.Compose([
#     transforms.Resize(1024),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
# ])



# # ---------- Custom Dataset ----------
# class SingleLabelDataset(Dataset):
#     def __init__(self, dataframe, img_root, transform=None):
#         self.df = dataframe.reset_index(drop=True)
#         self.root = Path(img_root)
#         self.transform = transform

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]
#         img = Image.open(self.root / row.file_name).convert("RGB")
#         label = torch.tensor(int(row.grand_id), dtype=torch.long)
#         if self.transform:
#             img = self.transform(img)
#         return img, label

# # ---------- Create Datasets & DataLoaders ----------
# train_ds = SingleLabelDataset(train_df, images_dir, transform=train_transform)
# test_ds  = SingleLabelDataset(test_df,  images_dir, transform=test_transform)

# train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=4)
# test_loader  = DataLoader(test_ds,  batch_size=8, shuffle=False, num_workers=4)

# print("✓ DataLoaders ready: 1024-pixel images without center crop.")

from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
import torch
from torch.utils.data import TensorDataset, DataLoader
from torchvision import transforms

# ---------- Paths ----------
data_dir   = Path("data")
images_dir = data_dir
csv_file   = data_dir / "image_labels.csv"

# ---------- Train / Test split ----------
df = pd.read_csv(csv_file)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
print(f"Train: {len(train_df)}  Test: {len(test_df)}")

# ---------- Transforms ----------
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.Resize((1024, 1024)),     # shortest side = 1024, keep aspect ratio
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((1024, 1024)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

# ---------- Load all images once and wrap in TensorDataset ----------
def make_tensor_dataset(frame, transform):
    imgs, labels = [], []
    for _, row in frame.iterrows():
        img = Image.open(images_dir / row.file_name).convert("RGB")
        img = transform(img)
        imgs.append(img)
        labels.append(int(row.grand_id))
    return TensorDataset(torch.stack(imgs),
                         torch.tensor(labels, dtype=torch.long))

train_ds = make_tensor_dataset(train_df, train_transform)
test_ds  = make_tensor_dataset(test_df,  test_transform)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=8, shuffle=False)

print("✓ DataLoaders ready for training")

Train: 1200  Test: 300
✓ DataLoaders ready for training


In [31]:
class CNN(nn.Module):
	def __init__(self, in_channels: int, num_classes: int = 10):
			super(CNN, self).__init__()
			# ---- Convolutional feature extractor ----
			self.conv1 = nn.Conv2d(in_channels, 8, kernel_size=3, stride=1, padding=1)
			self.pool  = nn.MaxPool2d(kernel_size=2, stride=2)
			self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)

			# ---- Global pooling + classifier ----
			# Always outputs (batch, 16, 1, 1) no matter the input H×W
			self.gap = nn.AdaptiveAvgPool2d((1, 1))
			self.fc1 = nn.Linear(16, num_classes)

	def forward(self, x):
			x = F.relu(self.conv1(x))
			x = self.pool(x)
			x = F.relu(self.conv2(x))
			x = self.gap(x)                # -> (batch, 16, 1, 1)
			x = torch.flatten(x, 1)        # -> (batch, 16)
			x = self.fc1(x)                # logits
			return x

In [24]:
if torch.backends.mps.is_available():
    device = torch.device("mps")   # Use Apple Silicon GPU
else:
    device = torch.device("cpu")   
    

print(device)

mps


In [29]:
print(train_df['grand_id'].value_counts())

grand_id
0    618
4    188
2    145
3    125
1     45
7     35
9     25
6     15
8      3
5      1
Name: count, dtype: int64


In [26]:
print(test_df['grand_id'].value_counts())


grand_id
0    164
4     47
3     34
2     31
7     10
1      8
9      3
6      3
Name: count, dtype: int64


In [41]:
from torch.utils.data import Subset

# small_train = Subset(train_ds, range(50))
# small_loader = DataLoader(small_train, batch_size=4, shuffle=True)


model = CNN(in_channels=3, num_classes=10).to(device)

class_counts = train_df['grand_id'].value_counts().sort_index()

# Inverse-frequency weighting: rarer classes get higher weight
weights = 1.0 / class_counts

# Normalise so average weight is 1.0 (optional, but keeps loss scale stable)
weights = weights / weights.mean()

class_weights = torch.tensor(weights.values, dtype=torch.float32).to(device)

print("Class counts:\n", class_counts)
print("Class weights:\n", class_weights)



criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) #weight_decay=1e-4

epochs = 60
for epoch in range(epochs):
    # ------------------ TRAIN ------------------
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)

    for imgs, labels in pbar:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        logits = model(imgs)
        loss   = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        _, predicted = logits.max(1)
        correct += (predicted == labels).sum().item()
        total   += labels.size(0)

        pbar.set_postfix({
            "loss": f"{running_loss/total:.4f}",
            "acc":  f"{100*correct/total:.2f}%"
        })

    train_acc  = 100 * correct / total
    train_loss = running_loss / total

    # ------------------ VALIDATION ------------------
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = outputs.max(1)
            val_correct += (preds == labels).sum().item()
            val_total   += labels.size(0)

    val_acc = 100 * val_correct / val_total

    print(f"Epoch {epoch+1:03d} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Train Acc: {train_acc:.2f}% | "
          f"Val Acc: {val_acc:.2f}%")

Class counts:
 grand_id
0    618
1     45
2    145
3    125
4    188
5      1
6     15
7     35
8      3
9     25
Name: count, dtype: int64
Class weights:
 tensor([0.0107, 0.1469, 0.0456, 0.0529, 0.0352, 6.6110, 0.4407, 0.1889, 2.2037,
        0.2644], device='mps:0')


Epoch 001 | Train Loss: 2.3355 | Train Acc: 7.67% | Val Acc: 3.67%


Epoch 002 | Train Loss: 2.1776 | Train Acc: 9.33% | Val Acc: 0.00%


Epoch 003 | Train Loss: 1.7857 | Train Acc: 12.92% | Val Acc: 0.00%


Epoch 004 | Train Loss: 1.3701 | Train Acc: 16.00% | Val Acc: 0.00%


...

Epoch 020 | Train Loss: 1.1698 | Train Acc: 16.25% | Val Acc: 0.67%


Epoch 021 | Train Loss: 1.1305 | Train Acc: 16.58% | Val Acc: 0.33%


Epoch 022 | Train Loss: 1.1695 | Train Acc: 18.83% | Val Acc: 0.33%


Epoch 023 | Train Loss: 1.0734 | Train Acc: 21.17% | Val Acc: 0.33%


KeyboardInterrupt: 